In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule

replace success!


In [14]:
class TestModel(torch.nn.Module):
  
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp1 = torch.nn.Linear(input_dim, output_dim)
    self.mlp1.weight.data.copy_(torch.zeros_like(self.mlp1.weight.data))
    self.mlp2 = torch.nn.Linear(output_dim,1)
    
  def forward(self,x):
    return self.mlp2(self.mlp1(x))



In [15]:
def gen_pattern_replace_and_matcher_for_linear(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name
                                                ):
  env  = utils.get_env(traced)
  target_node = env[target_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node_name)
  def _match(match,ori,pat):
    selected_node = None
    for node in pat.nodes:
      if node.op == "call_module" :
        selected_node = node
    return match.nodes_map[selected_node].name == target_node_name
  
  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.mlp = torch.nn.Linear(1, 1)
    def forward(self,x):
      return self.mlp(x)
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      target_weight = target_node_mod.weight
      target_bias = target_node_mod.bias
      redency_weight = target_weight[:,redency_part_slice[1]]
      self.redency_linear = torch.nn.Linear(redency_weight.shape[0],redency_weight.shape[1])
      self.redency_linear.weight.data = redency_weight
      self.redency_linear.bias.data = target_bias
      
      unredency_weight = target_weight[:,unredency_part_slice[1]]
      self.unredency_linear = torch.nn.Linear(unredency_weight.shape[0],unredency_weight.shape[1],bias=False)
      self.unredency_linear.weight.data = unredency_weight

    def forward(self,x):
      redency_part = x[redency_part_slice]
      unredency_part = x[unredency_part_slice]
      return self.redency_linear(redency_part) + self.unredency_linear(unredency_part)
    
  
  return PatternClass(),ReplacementClass(),_match

In [18]:
linear_func_model = symbolic_trace(TestModel(300,100))
linear_func_model.print_readable()

class TestModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        mlp1 = self.mlp1(x);  x = None
        mlp2 = self.mlp2(mlp1);  mlp1 = None
        return mlp2
        


'class TestModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        mlp1 = self.mlp1(x);  x = None\n        mlp2 = self.mlp2(mlp1);  mlp1 = None\n        return mlp2\n        '

In [17]:
sample = torch.ones((4096,300))
modify_mod = GraphModule(linear_func_model, linear_func_model.graph)

torch.onnx.export(modify_mod,sample,f'linear.onnx')

In [8]:
linear_func_model.graph.print_tabular()

opcode       name    target    args     kwargs
-----------  ------  --------  -------  --------
placeholder  x       x         ()       {}
call_module  mlp1    mlp1      (x,)     {}
call_module  mlp2    mlp2      (mlp1,)  {}
output       output  output    (mlp2,)  {}


In [19]:
pattern,replace,match = gen_pattern_replace_and_matcher_for_linear(linear_func_model,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      "mlp1")


In [20]:
matches = subgraph_rewriter.replace_pattern_with_filters(linear_func_model, pattern, replace,[match])

In [21]:
sample = torch.ones((4096,300))
modify_mod = GraphModule(linear_func_model, linear_func_model.graph)

torch.onnx.export(modify_mod,sample,f'linear_modify.onnx')

In [13]:
linear_func_model.print_readable()

class TestModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        getitem = x[(0, slice(None, 10, None))]
        getitem_1 = x[(slice(None, None, None), slice(10, None, None))];  x = None
        redency_linear = self.redency_linear(getitem);  getitem = None
        unredency_linear = self.unredency_linear(getitem_1);  getitem_1 = None
        add = redency_linear + unredency_linear;  redency_linear = unredency_linear = None
        mlp2 = self.mlp2(add);  add = None
        return mlp2
        


'class TestModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        getitem = x[(0, slice(None, 10, None))]\n        getitem_1 = x[(slice(None, None, None), slice(10, None, None))];  x = None\n        redency_linear = self.redency_linear(getitem);  getitem = None\n        unredency_linear = self.unredency_linear(getitem_1);  getitem_1 = None\n        add = redency_linear + unredency_linear;  redency_linear = unredency_linear = None\n        mlp2 = self.mlp2(add);  add = None\n        return mlp2\n        '

In [ ]:
class TestModel(torch.nn.Module):
  
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp1 = torch.nn.Linear(input_dim, output_dim)
    self.mlp1.weight.data.copy_(torch.zeros_like(self.mlp1.weight.data))
    self.mlp2 = torch.nn.Linear(output_dim,1)
    
  def forward(self,x):
    return self.mlp2(self.mlp1(x))